## LightweightMMM 에 Gemini 적용하기

by JeongMin Kwon (cojette@gmail.com)

LightweightMMM을 사람들이 더 손쉽게 활용하고 바로 마케팅에 적용할 수 있도록 일부 요소에 Gemini API를 적용하여 보다 쉬운 형태의 MMM(Marketing Mix Model)을 만듭니다.

* LightweightMMM: https://github.com/google/lightweight_mmm
* Gemini AI Studio : https://ai.google.dev/


In [2]:
## !pip install --upgrade pip
!pip3 install --upgrade git+https://github.com/google/lightweight_mmm.git
!pip3 install numpyro==0.13.1
import pandas as pd
from lightweight_mmm import preprocessing, lightweight_mmm, plot, optimize_media
import jax.numpy as jnp
from sklearn.metrics import mean_absolute_percentage_error


  Cloning https://github.com/google/lightweight_mmm.git to /var/tmp/pip-req-build-07xhfrz_
  Running command git clone --filter=blob:none --quiet https://github.com/google/lightweight_mmm.git /var/tmp/pip-req-build-07xhfrz_
  Resolved https://github.com/google/lightweight_mmm.git to commit 4406aaa77bddc5b0d73d31c6cf4f2ace03f3ffda
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached matplotlib-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4


2024-04-22 12:25:50.143135: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-04-22 12:25:50.144981: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-04-22 12:25:50.145011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [3]:
!pip3 install google-cloud
!pip3 install google-oauth

import glob
from google.cloud import bigquery
from google.oauth2 import service_account


credential = service_account.Credentials.from_service_account_file([GCP Key File]) ## Input your key


# GCP 클라이언트 객체 생성
client = bigquery.Client(credentials = credential,
                         project = credential.project_id)

## 데이터 가져옴
sql = """
SELECT
    *,
    CASE
    WHEN Date LIKE '% 2021' THEN
      PARSE_DATE('%b %d, %Y', REPLACE(Date, ' "', ''))
    WHEN Date LIKE '%/2021' THEN
      PARSE_DATE('%m/%d/%Y', Date)
    ELSE
      NULL END AS Ad_Date
   FROM adanalysis.campaigndata
    """

query_job = client.query(sql)

# 데이터프레임으로 변환
df = query_job.to_dataframe()
df.head()

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


,Date,Brand_Alias,Ad_group_alias,ASIN_SKU_Alias,Marketplace,Impressions,Clicks,Spend,Sales,Orders,Units,Advertised_Units_sold,Other_SKU_units_sold,Advertised_SKU_Sales,Other_SKU_sales,Ad_Date
0,"Nov 08, 2021",Brand 1,Brand 1 Ad Group 1,ASIN374,Walmart,1168,23,7.37,0.0,0,0,0,0,0.0,0.0,2021-11-08
1,"Nov 12, 2021",Brand 1,Brand 1 Ad Group 1,ASIN374,Walmart,0,0,0.00,0.0,0,0,0,0,0.0,0.0,2021-11-12
2,"Nov 13, 2021",Brand 1,Brand 1 Ad Group 13,ASIN374,Walmart,1712,12,2.18,0.0,0,0,0,0,0.0,0.0,2021-11-13
3,"Nov 14, 2021",Brand 1,Brand 1 Ad Group 13,ASIN374,Walmart,1014,4,0.70,0.0,0,0,0,0,0.0,0.0,2021-11-14
4,"Nov 19, 2021",Brand 1,Brand 1 Ad Group 13,ASIN374,Walmart,1308,17,3.11,0.0,0,0,0,0,0.0,0.0,2021-11-19


In [4]:
# 구글 생성 AI 연동 및 차트 이미지 처리를 위해 필요한 라이브러리를 가져옴
!pip3 install -U -q google-generativeai

import google.generativeai as genai

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests

import PIL.Image
import IPython.display
from IPython.display import Markdown

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


## Gemini for EDA 

LightWeightMMM에서는 따로 데이터를 확인하는 과정이 없으나, 데이터를 파악하는 과정은 데이터 분석에서 가장 중요합니다. 데이터를 확인한 후 필요시 일부 데이터는 필터링하거나 보완할 수 있습니다. 

여기서는 Gemini를 사용해서 모든 사용자가 데이터의 현황을 확인 가능하도록 자동으로 간단하게 EDA를 해서 문서 형태로 확인할 수 있도록 합니다. 

이 때 이 데이터가 MMM에 사용되는 것을 고려하여 해당 맥락에 맞춘 문서를 만들도록 합니다. 

In [5]:
API_KEY = [Gemini API Key] ## Input your key
genai.configure(api_key=API_KEY)

In [7]:

genaimodel = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt_parts = "This data has information about different ad groups that were set for brands in a few marketplaces like Walmart. It's from BigQuery.  This data are usually organized by Campaign, Ad Group (or Ad Set), and Ads. A campaign can have many ad groups that target different groups of people. For example, one ad group can be for people that subscribed to the e-mail list, another for people that visited sites. Then, inside each ad group, we have the ads that people in that group will see. We can have many ads, with different messages and formats (video vs image). We never really know which campaign or ad group will be the best before running. It’s normal to launch multiple ad groups and after a few days turn off the ones that are not reaching our goals (like paying too much for someone to install an app). This time, we have to analyze this data. It's very crucial for us. First of all, please Do a simple EDA with this data and find outlier rows that are unlikely to be utilized in the marketing mix model. Thanks in advance."

# 모델 설정
generation_config = {
  "temperature": 2,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192
}

response = genaimodel.generate_content([prompt_parts, df.to_csv()], generation_config=generation_config)



In [8]:
## 한글 번역

response2 = genaimodel.generate_content(["다음 내용을 한국어로 바꿔줘. ", response.text], generation_config=generation_config)

In [9]:
import textwrap

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## 마크다운으로 표시
to_markdown(response2.text)

> ## 광고 그룹 실적에 대한 탐색적 데이터 분석 (EDA)
> 
> 제공된 데이터를 기반으로, 잠재적 이상치에 중점을 둔 초기 EDA는 다음과 같습니다.
> 
> **요약 통계:**
> 
> *   **노출 수:** 광고 그룹당 평균 노출 수는 약 9,943회이고, 중앙값은 2,889회입니다. 이는 몇몇 높은 노출 수를 가진 광고 그룹의 영향으로 인해 분포가 왜곡되었을 가능성을 나타냅니다.
> *   **클릭 수:** 평균 클릭 수는 26회이지만, 중앙값은 9회에 불과하여 다시 한 번 오른쪽으로 치우친 분포를 암시합니다. 대부분의 광고 그룹은 클릭 수가 적고, 일부만이 많은 클릭 수를 유도하는 것으로 보입니다.
> *   **지출:** 평균 지출은 \$33.64이고, 중앙값은 \$12.82로, 역시 오른쪽으로 치우친 분포를 보입니다. 이는 대부분의 광고 그룹은 상대적으로 지출이 적지만, 일부는 상당히 높은 지출을 하는 것으로 해석할 수 있습니다.
> *   **판매 및 주문:** 이 두 지표는 평균값이 중앙값보다 훨씬 높아, 분포가 매우 치우쳐 있음을 나타냅니다. 몇몇 광고 그룹이 상당한 판매 및 주문을 유도하는 반면, 더 큰 비중은 최소한의 판매/주문을 생성하거나 아예 생성하지 않습니다.
> *   **판매된 단위 및 광고된 판매 단위:** 판매 및 주문과 마찬가지로, 판매된 단위와 광고된 판매 단위 모두 평균값이 중앙값보다 상당히 큽니다. 이는 판매 및 주문과 유사한 오른쪽으로 치우친 패턴을 암시합니다. 일부 광고 그룹은 많은 판매를 하지만, 많은 광고 그룹은 판매량이 매우 적거나 전혀 없습니다. 
> *   **기타 지표:** 기타 SKU 판매 단위, 광고된 SKU 판매 및 기타 SKU 판매는 판매 및 주문과 유사한 패턴을 보이는 것 같습니다. 이는 값이 매우 높은 이상치 행이 존재할 가능성을 나타냅니다.
> 
> **잠재적 이상치**:
> 
> 잠재적 이상치를 식별하기 위해 여러 가지 접근 방식을 고려할 수 있습니다.
> 
> 1.  **노출/클릭/지출이 0인 행:** 노출, 클릭 및 지출이 0이지만 판매 및 주문은 0이 아닌 행이 많이 있습니다. 이러한 행은 데이터 기록 오류 또는 직접적인 광고 참여 없이 발생한 판매(유기적 판매)를 나타낼 수 있습니다. 이를 추가로 조사하는 것이 좋습니다.
> 2.  **왜곡이 심한 지표:** 판매, 주문, 판매 단위와 같은 지표의 경우, 값이 75번째 백분위수보다 상당히 높거나 사분위수 범위의 3배를 초과하는 행을 잠재적 이상치로 간주해야 합니다. 이러한 성과가 높은 광고 그룹의 맥락을 파악하는 것이 중요합니다.
> 
> **추가 EDA 단계:**
> 
> 1.  **시각화:** 노출 수, 클릭 수, 지출, 판매, 주문과 같은 주요 지표에 대한 히스토그램 또는 상자 그림을 생성하여 분포를 더 잘 이해하고 이상치 행을 시각적으로 식별합니다. 
> 2.  **세분화:** Marketplace, Brand\_Alias, Ad\_group\_alias별로 실적을 분석하여 잠재적인 차이점을 파악하고 각 세그먼트 내에서 이상치를 식별합니다.
> 3.  **상관 관계:** 주요 지표 간의 상관 관계를 조사합니다. 예를 들어, 지출이 많을수록 판매/주문이 증가하는지 또는 특정 지출 임계값을 초과하면 수익이 감소하는지 확인합니다.
> 
> **이상치 활용:**
> 
> *   **맥락 조사:** 일부 광고 그룹에서 값이 매우 높거나 낮은 이유를 이해합니다. 특정 캠페인 설정, 타겟 잠재 고객, 계절성, 제품 특성 또는 외부 요인 때문일 수 있습니다. 
> *   **잠재적 제거:** 데이터 오류로 인해 이상치가 발생했거나 마케팅 믹스 모델과 관련이 없는 것으로 간주되는 경우, 일반적인 광고 그룹 실적을 더 정확하게 나타내기 위해 이상치를 제거하거나 조정하는 것을 고려합니다. 
> *   **모델링 고려 사항:** 마케팅 믹스 모델에서는 이상치에 강건한 분위 회귀 또는 강건 회귀와 같은 다양한 모델링 기술을 탐구하는 것이 좋을 수 있습니다.
> 
> 포괄적인 EDA를 수행하고 잠재적 이상치 행을 신중하게 분석하면 광고 그룹 실적에 대한 귀중한 통찰력을 얻고 마케팅 믹스 모델이 안정적인 기반 위에 구축되도록 할 수 있습니다.


## 데이터 정제

MMM에 사용할 수 있는 형태로 데이터를 만듭니다. 데이터를 다듬고, 데이터셋을 훈련 데이터와 검정 데이터로 구분합니다. 

In [10]:
agg_data = df.groupby(["Ad_Date", "Ad_group_alias"])[["Impressions","Spend", "Sales",]].sum()
agg_data = agg_data.drop(["Brand 1 Ad Group 12"], axis=0, level=1) # zero cost train

media_data_raw = agg_data['Impressions'].unstack().fillna(0)
costs_raw = agg_data['Spend'].unstack()
sales_raw = agg_data['Sales'].reset_index().groupby("Ad_Date").sum()


In [11]:
split_point = pd.Timestamp("2021-12-15") # 28 days to end of data

media_data_train = media_data_raw.loc[:split_point - pd.Timedelta(1,'D')]
media_data_test = media_data_raw.loc[split_point:]

target_train = sales_raw.loc[:split_point - pd.Timedelta(1,'D')]
target_test = sales_raw.loc[split_point:]

costs_train = costs_raw.loc[:split_point - pd.Timedelta(1,'D')].sum(axis=0).loc[media_data_train.columns]

In [12]:
import numpy as np

## Make random noises
organic_raw = pd.DataFrame({'organic_search': 0, 'organic_social': 0}, index=media_data_raw.index)
organic_raw['organic_search'] = sales_raw['Sales'].values/10 + np.random.randint(10000, 100000, organic_raw.shape[0])
organic_raw['organic_social'] = sales_raw['Sales'].values/10 + np.random.randint(10000, 100000, organic_raw.shape[0])

### 데이터 스케일링

LightweightMMM에서는 sklearn 기반의 내부의 Scaler를 제공합니다. 대부분의 경우 3~4개의 Scaler가 필요합니다. 미디어 데이터용 Scaler 하나, 타겟용 Scaler, 비용용 Scaler. 선택적으로 추가 기능을 추가하는 경우 추가 스케일러가 필요할 수 있습니다. LighweightMMM에서는 여러 군데에서 Scaler가 사용되고, 이를 사용해서 여러 단계에서 값을 조율할 수 있습니다.



In [13]:

## Divide Training data and test data with timestamp
split_point = pd.Timestamp("2021-12-15") # 28 days to end of data

media_data_train = media_data_raw.loc[:split_point - pd.Timedelta(1,'D')]
media_data_test = media_data_raw.loc[split_point:]


organic_data_train = organic_raw.loc[:split_point - pd.Timedelta(1,'D')]
organic_data_test = organic_raw.loc[split_point:]

target_train = sales_raw.loc[:split_point - pd.Timedelta(1,'D')]
target_test = sales_raw.loc[split_point:]

costs_train = costs_raw.loc[:split_point - pd.Timedelta(1,'D')].sum(axis=0).loc[media_data_train.columns]

## Scale data with mean()
media_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
organic_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
target_scaler = preprocessing.CustomScaler(
    divide_operation=jnp.mean)
cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)

### data transformation for model
media_data_train_scaled = media_scaler.fit_transform(media_data_train.values.astype(int))
organic_data_train_scaled = organic_scaler.fit_transform(organic_data_train.values)
target_train_scaled = target_scaler.fit_transform(target_train['Sales'].values.squeeze())
costs_scaled = cost_scaler.fit_transform(costs_train.values.astype(int))

media_data_test_scaled = media_scaler.transform(media_data_test.values.astype(int))
organic_data_test_scaled = organic_scaler.transform(organic_data_test.values)

media_names = media_data_raw.columns

## 모델링 

여기서는 adstock, hill_adstock, carryover 의 세 가지 모델을 제공합니다. 각각의 모델은 media별 사전 확률을 활용하는 수식과 계절성 활용 정도에 따라 구분됩니다. 

In [14]:


#adstock_models = ["adstock", "hill_adstock", "carryover"]
#degrees_season = [1,2,4]

adstock_models = ["carryover"]
degrees_season = [1]

for model_name in adstock_models:
  for degrees in degrees_season:
    mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
    mmm.fit(media=media_data_train_scaled,
            media_prior=costs_scaled,
            target=target_train_scaled,
            extra_features=organic_data_train_scaled,
            number_warmup=10,
            number_samples=1000,
            number_chains=1,
            degrees_seasonality=degrees,
            weekday_seasonality=True,
            seasonality_frequency=365,
            seed=1)

    prediction = mmm.predict(
    media=media_data_test_scaled,
    extra_features=organic_data_test_scaled,
    target_scaler=target_scaler)
    p = prediction.mean(axis=0)

    mape = mean_absolute_percentage_error(target_test['Sales'].values, p)
    print(f"model_name={model_name} degrees={degrees} MAPE={mape} samples={p[:3]}")

sample: 100%|██████████| 1010/1010 [10:03<00:00,  1.67it/s, 1023 steps of size 2.53e-03. acc. prob=0.91]


model_name=carryover degrees=1 MAPE=0.10130103167338399 samples=[30045.574 28512.982 27757.018]


In [15]:
!pip install matplotlib==3.1.3

plt = plot.plot_response_curves(
    media_mix_model=mmm, target_scaler=target_scaler, seed=1)
fig = plt.get_figure()
fig.savefig("plot_res_curve.png", bbox_inches = "tight")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Using cached matplotlib-3.1.3-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.6.1
    Uninstalling matplotlib-3.6.1:
      Successfully uninstalled matplotlib-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.18.0 requires matplotlib>=3.5, but you have matplotlib 3.1.3 which is incompatible.
explainable-ai-sdk 1.3.3 requires matplotlib>=3.2.2, but you have matplotlib 3.1.3 which is incompatible.
lightweight-mmm 0.1.9 requires matplotlib==3.6.1, but you have matplotlib 3.1.3 which is incompatible.
ydata-profiling 4.6.0 requires matplotlib<=3.7.3,>=3.2, but you have matplotlib 3.1.3 which is incompatible.


In [16]:
!python -m pip3 uninstall matplotlib
!pip3 install matplotlib
!pip3 install arviz

import arviz

/opt/conda/bin/python: No module named pip3


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Using cached matplotlib-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
Using cached matplotlib-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightweight-mmm 0.1.9 requires matplotlib==3.6.1, but you have matplotlib 3.8.4 which is incompatible.
ydata-profiling 4.6.0 requires matplotlib<=3.7.3,>=3.2, but you have matplotlib 3.8.4 which is incompatible.


In [17]:
media_data_train.head()

Ad_group_alias,Brand 1 Ad Group 1,Brand 1 Ad Group 10,Brand 1 Ad Group 11,Brand 1 Ad Group 13,Brand 1 Ad Group 2,Brand 1 Ad Group 3,Brand 1 Ad Group 4,Brand 1 Ad Group 5,Brand 1 Ad Group 6,Brand 1 Ad Group 7,Brand 1 Ad Group 8,Brand 1 Ad Group 9,Brand 2 Ad Group 1,Brand 2 Ad Group 2,Brand 2 Ad Group 3,Brand 2 Ad Group 4,Brand 2 Ad Group 5,Brand 1 Ad Group 14,Brand 2 Ad Group 6
Ad_Date,,,,,,,,,,,,,,,,,,,
2021-10-17,3317,33,50966,9519,40265,0,142704,55,94,1054,248,23321,276437,424883,135366,41905,9871,0,0
2021-10-18,1501,30,39424,14015,18545,0,389663,53,121,1669,283,28321,313907,492277,119675,46550,19920,50619,0
2021-10-19,771,63,236572,16543,13078,0,810070,58,122,1795,245,26105,296159,475287,191075,0,9253,43019,0
2021-10-20,958,70,277649,24590,10743,0,821705,60,111,948,234,0,314952,641517,98214,61017,18669,45642,0
2021-10-21,1841,59,251202,14236,10799,0,1241204,72,150,739,394,0,392621,412898,88922,121214,12977,69165,0


## 결과 해석 및 모델 설명

기본으로 제공하는 그래프도 베이지안 사전 지식이나 분포 사전 지식 등이 필요하고 모델에 대한 이해가 있어야 하므로 그래프를 기반으로 Gemini에게 설명을 맡겨서 그래프와 함께 내용을 같이 확인할 수 있도록 합니다.


plt = plot.plot_prior_and_posterior(media_mix_model=mmm)
fig = plt.get_figure()


In [19]:
fig.savefig("plot_media_posterior.png", bbox_inches = "tight")

In [20]:
media_effect_hat, roi_hat = mmm.get_posterior_metrics()
plt = plot.plot_bars_media_metrics(metric = media_effect_hat, channel_names=media_names)
fig = plt.get_figure()
fig.savefig("plot_media_hat.png", bbox_inches = "tight")
plt = plot.plot_bars_media_metrics(metric = roi_hat, channel_names=media_names)
fig = plt.get_figure()
fig.savefig("plot_roi_hat.png", bbox_inches = "tight")

In [21]:
plt = plot.plot_response_curves(
    media_mix_model=mmm, target_scaler=target_scaler, seed=1)
fig = plt.get_figure()
fig.savefig("plot_res_curve.png", bbox_inches = "tight")

In [22]:
from pathlib import Path
import hashlib

uploaded_files = []
def upload_if_needed(pathname: str) -> list[str]:
  path = Path(pathname)
  hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
  try:
    existing_file = genai.get_file(name=hash_id)
    return [existing_file]
  except:
    pass
  uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
  return [uploaded_files[-1]]

#   img1, img2, img3, img4,  , img2, img3, img4, 

prompt_parts = [
  "These graphs are from marketing mix model result.", 
  "First graph shows an visualize the prior and posterior distributions for every model parameter at once. Note that we use a kernel density estimator to smooth these distributions for easier interpretability; you may need to adjust the bandwidth of the smoother depending on the shapes of your distributions.", 
  "Second plot is also about media effect metrics. Third plot is the ROI plot. These plots take into account not only the media effect but how much it costs to get this effect. Another vital question we can solve with MMMs is how each media channel behaves individually as we invest more in it. ",
  "For that we plot the curve response of all media channels. It is the fourth plot. ", 
  " I need to use this graph to write a report to help us make decisions about our marketing advertising budget.",
  *upload_if_needed('plot_media_posterior.png'),
  *upload_if_needed("plot_media_hat.png"),
  *upload_if_needed("plot_roi_hat.png"),
  *upload_if_needed("plot_res_curve.png"),
  "Please be as helpful as possible in describing your interpretation of the graph and what you can do with it. Thanks in advance.  ",
]
response = genaimodel.generate_content(prompt_parts, generation_config=generation_config)
to_markdown(response.text)
for uploaded_file in uploaded_files:
  genai.delete_file(name=uploaded_file.name)


In [23]:
response2 = genaimodel.generate_content(["다음 내용을 한국어로 바꿔줘. ", response.text], generation_config=generation_config)
to_markdown(response2.text)

> ## 마케팅 믹스 모델 결과 해석:
> 
> 제공해주신 그래프를 기반으로 마케팅 예산 할당에 대한 해석과 잠재적인 조치를 제시합니다. 
> 
> **1. 반응 곡선:**
> 
> * **채널 0, 1, 2:** 이 채널들은 강한 양의 반응을 보입니다. 즉, 지출이 증가하면 결과 (전환, 매출 등) 도 증가한다는 것을 의미합니다. 이러한 고성과 채널에 더 많은 예산을 할당하는 것을 고려해 보세요. 
> * **채널 12:** 이 채널은 수확 체감의 법칙을 보여줍니다. 처음에는 효과적이지만 지출이 증가함에 따라 영향력이 감소합니다. 비용 대비 효과를 평가하고 적절한 예산을 고려하세요.
> * **채널 15, 17:** 이 채널들은 거의 또는 전혀 반응을 보이지 않습니다. 실적이 저조한 이유를 분석하고 이러한 채널에 대한 지출을 줄이거나 전략을 수정하는 것을 고려해 보세요. 
> * **기타 채널:** 대부분의 채널은 비교적 선형적인 반응을 보입니다. ROI와 전반적인 마케팅 목표에 따라 예산을 유지하거나 조정하세요. 
> 
> **2. 미디어 효과 지표:**
> 
> * **브랜드 1 광고 그룹 10, 브랜드 1 광고 그룹 13, 브랜드 2 광고 그룹 1:** 이 그룹들은 가장 높은 효과를 보이며, 성공적인 타겟팅과 메시지를 나타낼 가능성이 있습니다. 이러한 그룹이 성공한 요인을 분석하고 다른 그룹에도 전략을 적용하세요. 
> * **브랜드 1 광고 그룹 5, 6, 7, 브랜드 2 광고 그룹 2, 3, 4, 5:** 이 그룹들은 낮거나 중간 정도의 효과를 보입니다. 실적을 평가하여 개선 영역을 파악하거나 예산 할당을 줄이는 것을 고려하세요. 
> 
> **3. ROI 플롯 (없음):**
> 
> * **각 채널에 대한 ROI 분석:** 수익률이 높은 채널을 파악하고 예산 할당에서 우선순위를 부여하세요. 
> * **ROI가 낮은 채널 평가:** 실적이 저조한 이유를 조사하고, 전략 수정, 캠페인 최적화 또는 예산 재할당을 고려하세요. 
> 
> **실행 가능한 단계:**
> 
> 1. **고성과 채널 우선순위 지정:** 예산의 상당 부분을 채널 0, 1, 2, 그리고 강력한 반응과 높은 ROI를 보이는 다른 채널에 할당하세요. 
> 2. **저성과 채널 최적화:** 15, 17과 같이 실적이 낮은 채널의 이유를 분석하고 파악하여 지출 감소 또는 대체 전략 테스트를 고려하세요.
> 3. **성공적인 광고 그룹 확장:**  효과 지표가 높은 광고 그룹에서 얻은 교훈을 바탕으로 효과 지표가 낮은 다른 그룹에도 유사한 전략을 적용하세요.
> 4. **지속적인 모니터링 및 조정:** 마케팅 믹스 모델 결과를 정기적으로 검토하고 실적 및 시장 변화에 따라 예산 할당을 조정하세요. 
> 
> **추가 고려 사항:**
> 
> * **외부 요인:** 계절성, 경쟁사 활동, 경제 상황 등 외부 요인이 채널 실적에 미치는 영향을 평가하세요.
> * **시너지 효과:**  다른 채널이 어떻게 함께 작동하는지 분석하고 예산을 할당할 때 잠재적인 시너지 효과를 고려하세요.
> * **브랜드 목표:** 예산 할당을 전반적인 브랜드 목표 및 마케팅 목표와 일치시키세요. 
> 
> **마케팅 믹스 모델에서 얻은 통찰력을 활용하고 이러한 요소들을 고려함으로써, 데이터 기반 의사 결정을 내려 마케팅 예산을 최적화하고 더 나은 캠페인 결과를 달성할 수 있습니다.** 


## 예산 최적화

주어진 효과를 모델에서 파악한 정도에 따라서 최적으로 채널별 예산을 분배함

In [27]:
prices = jnp.ones(mmm.n_media_channels)
n_time_periods = 50
budget = prices * media_data_raw.mean(axis=0)* n_time_periods

In [28]:
budget.values

array([1.92058553e+05, 1.13815789e+03, 3.61940461e+06, 9.96236842e+05,
       1.27609408e+06, 1.32414474e+04, 2.13091737e+07, 2.11907895e+03,
       2.05361842e+04, 2.95625000e+04, 2.74296053e+04, 4.54442763e+05,
       1.71435796e+07, 2.83338428e+07, 5.25976447e+06, 1.02199868e+06,
       1.00021053e+06, 3.57528947e+05, 8.96907895e+03])

In [39]:
# Run optimization with the parameters of choice.
solution, kpi_without_optim, previous_media_allocation = optimize_media.find_optimal_budgets(
    n_time_periods=n_time_periods,
    media_mix_model=mmm,
#    extra_features=extra_features_scaler.transform(extra_features_test)[:n_time_periods],
    budget= sum(budget.values), 
    prices=prices,
    media_scaler=media_scaler,
    target_scaler=target_scaler,
    max_iterations = 10,
    seed=1)

Iteration limit reached    (Exit mode 9)
            Current function value: -1251056.5948636667
            Iterations: 10
            Function evaluations: 390
            Gradient evaluations: 10


In [40]:
optimal_buget_allocation = prices * solution.x
optimal_buget_allocation

Array([1.8141009e+05, 1.5976271e+03, 3.5399262e+06, 9.6178794e+05,
       1.2558705e+06, 1.6166907e+04, 2.2025928e+07, 2.3277966e+03,
       2.2146145e+04, 3.0611891e+04, 2.5923617e+04, 4.4037706e+05,
       1.7143188e+07, 2.7503890e+07, 5.2502645e+06, 1.2654550e+06,
       9.4129088e+05, 4.5191903e+05, 7.2498643e+03], dtype=float32)

In [41]:
# similar renormalization to get previous budget allocation
previous_budget_allocation = prices * previous_media_allocation
previous_budget_allocation

Array([1.8144233e+05, 1.3068665e+03, 3.5400885e+06, 9.6188712e+05,
       1.2560151e+06, 1.5516023e+04, 2.2026096e+07, 1.9041487e+03,
       2.1957184e+04, 3.0444746e+04, 2.5856990e+04, 4.4047891e+05,
       1.7143354e+07, 2.7504056e+07, 5.2504270e+06, 1.2656099e+06,
       9.4143306e+05, 4.5207431e+05, 7.3828384e+03], dtype=float32)

In [49]:
# Both these values should be very close in order to compare KPI
budget, optimal_budget_allocation.sum()

NameError: name 'optimal_budget_allocation' is not defined

## 모델 저장

모델을 저장해두고 재활용할 수 있습니다. 

In [44]:
from lightweight_mmm import utils

# We can use the utilities for saving models to disk.
file_path = "media_mix_model.pkl"
utils.save_model(media_mix_model=mmm, file_path=file_path)

## 예산 최적화에 대한 비교 설명

예산 최적화 결과와 이전 예산안을 비교한 후에 이에 대해서 자동으로 파악을 할 수 있습니다.
(여기서 사용한 데이터에는 적합하지 않아서 생략함) 

In [ ]:
','.join(budget.index)
#np.array2string(budget.index, separator=',')[1: len(np.array2string(budget.index, separator=','))-1]

'Brand 1 Ad Group 1,Brand 1 Ad Group 10,Brand 1 Ad Group 11,Brand 1 Ad Group 13,Brand 1 Ad Group 2,Brand 1 Ad Group 3,Brand 1 Ad Group 4,Brand 1 Ad Group 5,Brand 1 Ad Group 6,Brand 1 Ad Group 7,Brand 1 Ad Group 8,Brand 1 Ad Group 9,Brand 2 Ad Group 1,Brand 2 Ad Group 2,Brand 2 Ad Group 3,Brand 2 Ad Group 4,Brand 2 Ad Group 5,Brand 1 Ad Group 14,Brand 2 Ad Group 6'

In [ ]:
## budget이 어려운 경우에 설명. 여기서는 거의 비슷한 결과가 나왔으므로 생략함. 

prompt_parts = [
  "Here's the optimal budget allocation using this marketing mix model, and the second value is the existing budget allocation, and the third value is the ad group name index for each of these. Compare the two and write a nice report about what you think we should do in the future. And then I'll do what you want. Thank you very much.    ",
    np.array2string(optimal_buget_allocation, separator=',')[1: len(np.array2string(optimal_buget_allocation, separator=','))-1],
    np.array2string(previous_budget_allocation, separator=',')[1: len(np.array2string(optimal_buget_allocation, separator=','))-1] ,
    ','.join(budget.index)
]
response = genaimodel.generate_content(prompt_parts, generation_config=generation_config)
to_markdown(response.text)

> ## Analysis of Optimal vs. Existing Budget Allocation for Marketing Campaigns
> 
> This report analyzes the differences between the recommended optimal budget allocation and the existing budget allocation for your marketing campaigns.  It identifies areas of improvement and suggests future strategies.
> 
> **Observations:**
> 
> * **Significant discrepancies exist between optimal and current budgets for several ad groups.** This indicates potential inefficiencies in budget allocation, where some campaigns might be over- or under-funded compared to their predicted impact.
> * **Both Brand 1 and Brand 2 exhibit these discrepancies.** This suggests the issue is not brand-specific but applies across campaigns.
> 
> **Recommendations:**
> 
> 1. **Review Ad Group Performance:** Deep dive into the performance metrics (impressions, clicks, conversions) of ad groups with significant budget discrepancies. This analysis can identify underperforming ad groups that might be receiving excessive budgets or high-performing ones needing further investment.
> 2. **Consider Gradual Budget Adjustments:** Instead of drastic changes, consider incrementally adjusting budgets towards the optimal allocation. This approach allows for monitoring performance and making necessary refinements along the way. 
> 3. **A/B Testing for Validation:**  Implement A/B testing to compare the performance of current budget allocation against the optimal model's recommendations. This will provide empirical data to support decision making. 
> 4. **Investigate Underlying Factors:**  Explore potential causes for the discrepancies.  Are there specific targeting parameters, ad creatives, or landing page experiences influencing performance?  Understanding these factors can further refine budget allocation and campaign optimization strategies.
> 5. **Refine the Model (if necessary):** While the marketing mix model offers valuable insights, consider refining it further by incorporating additional variables or data sources that could impact performance predictions.  
> 
> **Additional Considerations:**
> 
> * **Seasonality and Trends:** Evaluate seasonality and market trends when adjusting budgets, as some campaigns might require higher spending during specific periods. 
> * **Competitor Analysis:**  Analyze competitor activity to understand their strategies and adjust budgets accordingly. 
> * **Campaign Goals:** Align budget allocation with specific campaign goals, such as brand awareness, lead generation, or sales conversions.
> 
> **Conclusion:**
> 
> By aligning your budget allocation with the optimal recommendations and employing a data-driven approach, you can improve marketing campaign performance and maximize return on investment.  Continuously monitor results and refine strategies for sustained success.


In [ ]:
response2 = genaimodel.generate_content(["다음 내용을 한국어로 바꿔줘. ", response.text], generation_config=generation_config)
to_markdown(response2.text)

> ## 마케팅 캠페인 최적 예산 할당 vs. 기존 예산 할당 분석
> 
> 이 보고서는 마케팅 캠페인에 대한 권장 최적 예산 할당과 기존 예산 할당의 차이점을 분석하고, 개선 영역을 파악하며 향후 전략을 제안합니다.
> 
> **관찰 결과:**
> 
> * **여러 광고 그룹에서 최적 예산과 현재 예산 사이에 상당한 차이가 있습니다.**  이는 예산 할당에 비효율성이 있을 수 있음을 나타내며, 일부 캠페인은 예상되는 효과에 비해 과소 또는 과대 투자되었을 수 있습니다.
> * **브랜드 1과 브랜드 2 모두 이러한 차이를 보입니다.**  이는 문제가 특정 브랜드에 국한되지 않고 캠페인 전반에 적용됨을 시사합니다.
> 
> **권장 사항:**
> 
> 1. **광고 그룹 실적 검토:**  예산 차이가 큰 광고 그룹의 실적 지표(노출 수, 클릭 수, 전환 수)를 자세히 살펴보세요. 이 분석을 통해 과도한 예산을 받는 성과가 저조한 광고 그룹이나 추가 투자가 필요한 성과가 우수한 광고 그룹을 식별할 수 있습니다.
> 2. **점진적 예산 조정 고려:**  급격한 변화보다는 최적 할당을 향해 예산을 점진적으로 조정하는 것을 고려하세요. 이 접근 방식을 통해 성과를 모니터링하고 필요한 조정을 진행할 수 있습니다.
> 3. **A/B 테스트를 통한 검증:**  A/B 테스트를 통해 현재 예산 할당과 최적 모델의 권장 사항에 따른 성과를 비교하세요. 이를 통해 의사 결정을 뒷받침할 실증적 데이터를 얻을 수 있습니다.
> 4. **근본적인 요인 조사:**  차이점의 잠재적 원인을 탐구하세요. 특정 타겟팅 매개변수, 광고 소재 또는 방문 페이지 경험이 성과에 영향을 미치는가?  이러한 요인을 이해하면 예산 할당 및 캠페인 최적화 전략을 더욱 개선할 수 있습니다.
> 5. **모델 개선(필요한 경우):** 마케팅 믹스 모델은 가치 있는 통찰력을 제공하지만, 성과 예측에 영향을 미칠 수 있는 추가 변수 또는 데이터 소스를 통합하여 더욱 개선할 수 있습니다.
> 
> **추가 고려 사항:**
> 
> * **계절성 및 트렌드:**  예산을 조정할 때 계절성 및 시장 트렌드를 평가하세요. 일부 캠페인은 특정 기간 동안 더 많은 지출이 필요할 수 있습니다.
> * **경쟁사 분석:**  경쟁사 활동을 분석하여 그들의 전략을 이해하고 예산을 적절하게 조정하세요.
> * **캠페인 목표:**  예산 할당을 브랜드 인지도, 리드 생성 또는 판매 전환과 같은 특정 캠페인 목표와 일치시키세요.
> 
> **결론:**
> 
> 예산 할당을 최적 권장 사항과 일치시키고 데이터 기반 접근 방식을 활용함으로써 마케팅 캠페인 성과를 개선하고 투자 수익률을 극대화할 수 있습니다. 지속적인 성과 모니터링과 전략 개선을 통해 지속적인 성공을 달성하세요. 
